# 第二讲 向量

## 数据结构分类：

- 线性结构
    - 序列
        - 向量(所有元素的逻辑顺序与其物理存放顺序一致;存放在连续的空间中)
        - 列表(除第一个元素,其余只有一个前继;除最后一个元素，其余只有一个后继)
- 半线性结构
- 非线性结构

## 向量

和数组不一样：向量是数组的抽象和泛化，需要通过秩访问，模板类实现，带有操作接口；

- 静态内存
- 动态内存

应用 = 接口 * 实现；

python中列表用[]表示，向量用numpy和array;

In [1]:
import numpy as np

np.set_printoptions(threshold=np.inf)  # 设置打印选项，显示所有元素

# 向量类定义
class Vector:
    def __init__(self, size):
        self.size = size
        self.data = np.zeros(size, dtype=int)  # 初始化向量为零

    def __getitem__(self, index):
        if 0 <= index < self.size:
            return self.data[index]
        else:
            raise IndexError("Index out of bounds")

    def __setitem__(self, index, value):
        if 0 <= index < self.size:
            self.data[index] = value
        else:
            raise IndexError("Index out of bounds")

    def __len__(self):
        return self.size

    def __str__(self):
        return str(self.data)

    def __repr__(self):
        return f"Vector(size={self.size}, data={self.data})"

    def resize(self, new_size):
        if new_size < self.size:
            self.data = self.data[:new_size]
        else:
            new_data = np.zeros(new_size, dtype=int)
            new_data[:self.size] = self.data
            self.data = new_data
        self.size = new_size

    def append(self, value):
        self.resize(self.size + 1)
        self.data[-1] = value

    def pop(self):
        if self.size == 0:
            raise IndexError("Pop from empty vector")
        value = self.data[-1]
        self.resize(self.size - 1)
        return value

    def clear(self):
        self.data.fill(0)

    def to_list(self):
        return self.data.tolist()

    @classmethod
    def from_list(cls, lst):
        vector = cls(len(lst))
        vector.data = np.array(lst, dtype=int)
        return vector 
    
# 测试向量类
if __name__ == "__main__":
    v = Vector(5)
    print("Initial vector:", v)

    for i in range(5):
        v[i] = i + 1
    print("After setting values:", v)

    print("Element at index 2:", v[2])

    v.append(6)
    print("After appending 6:", v)

    popped_value = v.pop()
    print("After popping value:", popped_value, "New vector:", v)

    v.clear()
    print("After clearing vector:", v)

    lst = [10, 20, 30]
    v_from_list = Vector.from_list(lst)
    print("Vector from list:", v_from_list)

    v.resize(3)
    print("After resizing to 3:", v)

    v.resize(10)
    print("After resizing to 10:", v)

    print("Length of vector:", len(v))
    print("Vector as list:", v.to_list())

    # 测试异常处理
    try:
        print(v[10])  # Index out of bounds
    except IndexError as e:
        print("Caught an exception:", e)

    try:
        v[10] = 100  # Index out of bounds
    except IndexError as e:
        print("Caught an exception:", e)

    try:
        v.pop()  # Pop from empty vector
    except IndexError as e:
        print("Caught an exception:", e)

    try:
        v.resize(0)  # Resize to zero
        print("After resizing to 0:", v)
    except Exception as e:
        print("Caught an exception during resize:", e)  


Initial vector: [0 0 0 0 0]
After setting values: [1 2 3 4 5]
Element at index 2: 3
After appending 6: [1 2 3 4 5 6]
After popping value: 6 New vector: [1 2 3 4 5]
After clearing vector: [0 0 0 0 0]
Vector from list: [10 20 30]
After resizing to 3: [0 0 0]
After resizing to 10: [0 0 0 0 0 0 0 0 0 0]
Length of vector: 10
Vector as list: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Caught an exception: Index out of bounds
Caught an exception: Index out of bounds
After resizing to 0: []


## 容量加倍策略
- 普通加倍(O(n)级别)
- 指数加倍(O(1)级别)

# 位图数据结构

In [2]:
def get_remainder_and_mod(num):
    remainder = num//31
    mod = num%31
    return remainder, 1<<mod   
    # 这个函数算出余数和模,并且将1向左推mod的数量，比如2这个数字就要推两位，
    # 刚好变为了100这个二进制数（十进制为4），这样理解了吧，被推的个数就是mod的大小，
    # 被推到变为100后，这个100的1不再会改变了（不能重复）
    # 变为100后，如果和1000这个数进行或运算，那么得到的结果就是1100，这时储存的就是两个数了
    # 之后和余数结合起来就成为了原始的数字了  
    # 注意这一大段中的10之类的都是二进制数字哈

a = [52,51,53,20,30,18,188,200,1000,10,15,20,18]  # 要储存的列表
arr = [0 for i in range(max(a)//31+1)] 
 # 用于储存的数组，根据最大的数字决定数组的大小，因为最大的数需要数组的最后一项来储存

for i in a:
    remainder, num_saved = get_remainder_and_mod(i)  
    print(f"Number: {i}, Remainder: {remainder}, Saved: {num_saved}")
    arr[remainder] = arr[remainder] | num_saved
    # 这里的|符号表示或运算，对两个十进制数字进行或运算实际上是先把数字都变成了二进制之后再进行的
    # 由于不同的数字可能有不同的余数，因此需要不同的四个字节来储存新的更大的数
    # 这里arr[remainder]的remainder就表示一个索引，而这个索引刚好也是余数的大小
    # 之后，或运算就表示了这个余数下，已经储存的值和将要储存的新值进行或运算，也就是开始的某个位为0的话
    # 如果传入的这个位为1，那么这个地方就变1，这个位置就代表一个新值了
    # 这里读者不懂的话可以多理解一下
# 这里生成的arr为
print(arr)
for i in range(max(a)+1):
    remainder, bit_index = get_remainder_and_mod(i)
    # print(f"Checking number: {i}, Remainder: {remainder}, Bit Index: {bit_index}")
    # 这里的i就是要检查的数字，remainder是余数，bit_index是1左移mod的结果
    # 也就是要检查的位
    if arr[remainder] & bit_index:  
        print(i)  
    # 由于之前存入的数字对应的位置已经存了1了，那么如果使用和运算的话应该是会返回1的
    # 这就是取回的方法

def check_number(bitmap, num):
    remainder, bit = get_remainder_and_mod(num)
    return bool(bitmap[remainder] & bit)

print(check_number(arr, 20)) # 输出: True
print(check_number(arr, 100)) # 输出: False


Number: 52, Remainder: 1, Saved: 2097152
Number: 51, Remainder: 1, Saved: 1048576
Number: 53, Remainder: 1, Saved: 4194304
Number: 20, Remainder: 0, Saved: 1048576
Number: 30, Remainder: 0, Saved: 1073741824
Number: 18, Remainder: 0, Saved: 262144
Number: 188, Remainder: 6, Saved: 4
Number: 200, Remainder: 6, Saved: 16384
Number: 1000, Remainder: 32, Saved: 256
Number: 10, Remainder: 0, Saved: 1024
Number: 15, Remainder: 0, Saved: 32768
Number: 20, Remainder: 0, Saved: 1048576
Number: 18, Remainder: 0, Saved: 262144
[1075086336, 7340032, 0, 0, 0, 0, 16388, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 256]
10
15
18
20
30
51
52
53
188
200
1000
True
False


In [3]:
class BitMap:
    def __init__(self, size):
        self.bitmap = create_bitmap(size)
        self.size = size

    def set(self, index):
        set_bit(self.bitmap, index)

    def clear(self, index):
        clear_bit(self.bitmap, index)

    def test(self, index):
        return test_bit(self.bitmap, index)

def create_bitmap(size):
    return bytearray((size + 7) // 8)

def set_bit(bitmap, index):
    bitmap[index // 8] |= 1 << (index % 8)

def clear_bit(bitmap, index):
    bitmap[index // 8] &= ~(1 << (index % 8))

def test_bit(bitmap, index):
    return (bitmap[index // 8] & (1 << (index % 8))) != 0

# 示例使用
bitmap = BitMap(100)
bitmap.set(10)
print(bitmap.test(10)) 
bitmap.clear(10)
print(bitmap.test(10)) 
print(bitmap.bitmap)  

True
False
bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


## 图像位图！


In [4]:
from PIL import Image

width, height = 100, 100
image = Image.new("RGB", (width, height), "white")

for x in range(width):
	for y in range(height):
		if (x % 10 == 0) and (y % 10 == 0):
			image.putpixel((x, y), (255, 0, 0)) # 红色像素

image.save("bitmap_example.bmp")

image = Image.open("bitmap_example.bmp")
image.show()
print("图像尺寸:", image.size)

resized_image = image.resize((50, 50))
resized_image.show()

gray_image = image.convert("L")
gray_image.show()

图像尺寸: (100, 100)
